In [1]:
# import internal .py modules
import file_path_management as fpath
import public_library as plib

2023-08-06 16:31:21 GM01X680 metapub.config[42655] WARNING NCBI_API_KEY was not set.


In [2]:
# import packages
import csv
import pandas as pd
import PyPDF2
import requests
import time
import os
import random
from requests.auth import HTTPProxyAuth
from bs4 import BeautifulSoup
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.firefox.service import Service
from selenium.common.exceptions import TimeoutException, WebDriverException, NoSuchElementException
import os
import re

In [3]:
# websites = ["PMC", "frontiersin", "europepmc", "biorxiv", "jneurosci", "orca.cardiff", "science", "thejns", "cambridge",
#                 "wiley", "ahajournals", "mdpi", "sciencedirect", "pnas", "nature", "cell", "eneuro", "physiology", "springer",
#                 "ieee", "plos", "jstage.jst", "biomedcentral", "jamanetwork", "psycnet.apa", "jnnp.bmj", "degruyter",
#                 "karger", "pure.mpg", "elifesciences", "neurology", "pubs.asahq", "sagepub", "ekja", "liebertpub", "lww",
#                 "tandfonline", "aspetjournals", "oup", "royalsocietypublishing", "psychiatryonline", "jpn", "open.bu.edu",
#                 "agro.icm", "lib.wfu", "mirasmart", "jstor"]
def PMC(url):
    os.environ['WDM_LOG'] = '0'
    options = Options()
    options.add_argument('--headless')
    
    # load the webpage
    error_label = 0
    while(error_label == 0):
        try:
            driver = webdriver.Firefox(options, service=Service(GeckoDriverManager().install()))
            driver.get(url)

            WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//p[text()='Consent']"))).click()
            
            error_label = 1
        except:
            print("Extracting content from:" + url + " failed, retrying... This might take longer than 5 minutes...")
            time.sleep(5*60)
            error_label = 0
    
    # extract information from loaded webpage
    try:
        my_elem = driver.find_elements(By.XPATH, "//*[contains(text(), 'doi.org') or contains(text(), 'DOI')]")
        doi = ""
        doi = str(my_elem.find_element(By.XPATH, "i").get_attribute("innerHTML"))
        abstract = ""
        keywords = ""
        intro = ""
        methods = ""
        pdf_link = ""
    except:
        doi = np.nan
    driver.quit()

    info = {
        "doi": doi,
        "abstract": abstract,
        "keywords": keywords,
        "introduction": intro,
        "methods": methods,
        "pdf_link": pdf_link
    }

    return info

def pmc(url):
    pass

def europepmc(url):
    pass

def biorxiv(url):
    pass

def jneurosci(url):
    pass

def orca_cardiff(url):
    pass

def science(url):
    pass

def thejns(url):
    pass

def cambridge(url):
    pass

def wiley(url):
    pass

def ahajournals(url):
    pass

def mdpi(url):
    pass

def ahajournals(url):
    pass

def sciencedirect(url):
    pass

def pnas(url):
    pass

def nature(url):
    pass

def cell(url):
    pass

def eneuro(url):
    pass

def physiology(url):
    pass

def springer(url):
    pass

def ieee(url):
    pass

def plos(url):
    pass

def jstage_jst(url):
    pass

def biomedcentral(url):
    pass

def jamanetwork(url):
    pass

def psycnet_apa(url):
    pass

def jnnp_bmj(url):
    pass

def degruyter(url):
    pass

def karger(url):
    pass

def pure_mpg(url):
    pass

def elifesciences(url):
    pass

def neurology(url):
    pass

def pubs_asahqa(url):
    pass

def sagepub(url):
    pass

# ekja
def ekja(url):
    pass

# liebertpub
def liebertpub(url):
    pass

# lww
def lww(url):
    pass

# tandfonline
def tandfonline(url):
    pass

# aspetjournals
def aspetjournals(url):
    pass

# oup
def aspetjournals(url):
    pass

# royalsocietypublishing
def royalsocietypublishing(url):
    pass

# psychiatryonline
def psychiatryonline(url):
    pass

# jpn
def jpn(url):
    pass

# open_bu_edu

# agro_icm

# lib_wfu

In [4]:
def extract_info_from_webpage(url, source):
    # {'www.cabdirect.org', 'books.google.de', 'www.taylorfrancis.com',  # all books
    websites = ["PMC", "frontiersin", "europepmc", "biorxiv", "jneurosci", "orca.cardiff", "science", "thejns", "cambridge",
                "wiley", "ahajournals", "mdpi", "sciencedirect", "pnas", "nature", "cell", "eneuro", "physiology", "springer",
                "ieee", "plos", "jstage.jst", "biomedcentral", "jamanetwork", "psycnet.apa", "jnnp.bmj", "degruyter",
                "karger", "pure.mpg", "elifesciences", "neurology", "pubs.asahq", "sagepub", "ekja", "liebertpub", "lww",
                "tandfonline", "aspetjournals", "oup", "royalsocietypublishing", "psychiatryonline", "jpn", "open.bu.edu",
                "agro.icm", "lib.wfu", "mirasmart", "jstor"]
    for website in websites:
        if website in source:
            url = plib.get_final_redirected_url(url)
            # Get the function name by replacing "." with "_" and use globals() to call it
            func_name = website.replace(".", "_")
            func = globals().get(func_name)
            if func:
                return func(url)
            else:
                raise Exception(f"Function {func_name} not defined.")
# --------------------start of test code--------------------
# websites = ["PMC", "frontiersin", "europepmc", "biorxiv", "jneurosci", "orca.cardiff", "science", "thejns", "cambridge",
#                 "wiley", "ahajournals", "mdpi", "sciencedirect", "pnas", "nature", "cell", "eneuro", "physiology", "springer",
#                 "ieee", "plos", "jstage.jst", "biomedcentral", "jamanetwork", "psycnet.apa", "jnnp.bmj", "degruyter",
#                 "karger", "pure.mpg", "elifesciences", "neurology", "pubs.asahq", "sagepub", "ekja", "liebertpub", "lww",
#                 "tandfonline", "aspetjournals", "oup", "royalsocietypublishing", "psychiatryonline", "jpn", "open.bu.edu",
#                 "agro.icm", "lib.wfu", "mirasmart", "jstor"]
# if len(websites) == len(set(websites)):
#     print("There are no duplicates in the list.")
# else:
#     print("There are duplicates in the list.")
# ---------------------end of test code---------------------

# --------------------start of test code--------------------
# url = "https://www.tandfonline.com/doi/abs/10.1080/01616412.1985.11739692"
# source = "www.tandfonline.com"
# info = extract_info_from_webpage(url, source)
# print(info)
# ---------------------end of test code---------------------

There are no duplicates in the list.


In [5]:
# get doi from url
def url2doi(url, source):
    url = str(url).strip()
    source = str(source).strip()
    info = extract_info_from_webpage(url, source) # dictionary
    doi = info["doi"]
    return doi
# --------------------start of test code--------------------
# url = "https://www.tandfonline.com/doi/abs/10.1080/01616412.1985.11739692"
# source = "www.tandfonline.com"
# doi = url2doi(url, source)
# print(doi)
# ---------------------end of test code---------------------